<a href="https://colab.research.google.com/github/Prach1/CS69002_9A_18CS60R30/blob/master/DL_Assign9b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pickle
import pandas as pd
import numpy as np
import io
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Train_20K.csv', sep='\t')
print(df.head())

                                                text  label
0  John Waters has given us a genuinely enjoyable...      1
1  This first two seasons of this comedy series w...      1
2  What an unfortunate mess is "Shiner." I wanted...      0
3  I'm not entirely sure Rob Schmidt qualifies as...      1
4  i wasn't sure whether to laugh or cry. Porrett...      0


In [5]:
df1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Test_5K.csv', sep='\t')
print(df1.head())

                                                text  label
0  as always this is an inaccurate picture of the...      0
1  Did the movie-makers even preview this before ...      0
2  Heavily re-edited and often confusing, the ori...      0
3  I notice that most of the people who think thi...      0
4  First of all, this is a low-budget movie, so m...      0


In [0]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)
print(type(text_labels))

text_reviews = [x.lower() for x in text_reviews]
#print(type(text_reviews))
#print(text_reviews[0])

In [0]:
text_test_reviews = df1['text'].astype(str).tolist()
text_test_labels = df1['label'].astype(int)
#print(type(text_labels))

text_test_reviews = [x.lower() for x in text_test_reviews]
#print(type(text_reviews))
#print(text_reviews[0])

In [0]:
import re

def preprocess(text):    
  text = re.sub(r'<br.*?>',' ',text)
  text = re.sub(r'[^\w\s]',' ',text) 
  text = re.sub(r'[0-9]+', ' ', text)
  
  return text
  

In [0]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

train_data = []
train_data_X = []
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for review in text_reviews:
  new_review = preprocess(review)
  train_data.append(new_review)
  
print(train_data[0])

for review in train_data:
  review = [lemmatizer.lemmatize(word) for word in review.split() if word not in stop and word != '' and len(lemmatizer.lemmatize(word)) > 1]
  train_data_X.append(review)
  
print(train_data_X[0])

In [0]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

test_data = []
test_data_X = []
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for review in text_test_reviews:
  new_review = preprocess(review)
  test_data.append(new_review)
  
print(test_data[0])

for review in test_data:
  review = [lemmatizer.lemmatize(word) for word in review.split() if word not in stop and word != '' and len(lemmatizer.lemmatize(word)) > 1]
  test_data_X.append(review)
  
print(test_data_X[0])

In [0]:
def generate_word_frequency(dataset):
  word_to_frequency = {}
 
  for sent in dataset:
    for word in sent:
      if word not in word_to_frequency:
        word_to_frequency[word] = 1
      else:
        word_to_frequency[word] += 1
              
  return word_to_frequency

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
 
  for word in dataset:
    if word not in word_to_ix:
      word_to_ix[word] = len(word_to_ix)
              
  word_to_ix['UNKNOWN'] = len(word_to_ix) 
              
  return word_to_ix


In [0]:
word_to_frequency = generate_word_frequency(train_data_X + test_data_X)
# for key in word_to_bigram_frequency:
#   print(key)
#   break

new_word_dataset = []

for word in word_to_frequency:
  if(word_to_frequency[word] >= 3):
    new_word_dataset.append(word)
        
print(len(word_to_frequency))
print(len(new_word_dataset))

word_to_ix = generate_word_ids(new_word_dataset)

print(len(word_to_ix))
print(word_to_ix['UNKNOWN'])

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [0]:
def make_bow_vector(sentence):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[word_to_ix['UNKNOWN']]+=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

# def make_target(label, label_to_ix):
#     return torch.LongTensor([label_to_ix[label]])
    #return torch.LongTensor(label)

In [0]:
train = []

for review in train_data_X:
  embedding = torch.autograd.Variable(make_bow_vector(review)).cuda()
  train.append(embedding)

In [0]:
#import pickle

test = []
for review in test_data_X:
  embedding = torch.autograd.Variable(make_bow_vector(review)).cuda()
  test.append(embedding)
  
# fp = open("test.pickle","wb")
# pickle.dump(test, fp)
# fp.close()

In [0]:
# data = []

# for index,review in enumerate(train):
#   data.append((review,text_labels[index]))
  
# print(data[0])

#label_to_tensor = torch.LongTensor(text_labels)

label_to_tensor = []

#print(text_labels)
text_labels1 = []

for label in text_labels:
  text_labels1.append([label])
  

#print(text_labels1)

for label in text_labels1:
  label_to_tensor.append(torch.tensor(label, dtype = torch.long))
  #print(label_to_tensor)
  
#print(label_to_tensor)

#for label in label_to_tensor:
  

In [0]:
label_to_test_tensor = []

text_test_labels1 = []

for label in text_test_labels:
  text_test_labels1.append([label])

for label in text_test_labels1:
  label_to_test_tensor.append(torch.tensor(label, dtype = torch.long))
  
#print(label_to_test_tensor)
# fp = open("test.pickle","wb")
# pickle.dump(label_to_test_tensor, fp)
# fp.close()

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.device('cuda') 

In [0]:
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [0]:
import time

# the training loop
for epoch in range(10):
  start = time.time()
  for index, instance in enumerate(train):
    bow.zero_grad()
    label = Variable(label_to_tensor[index]).cuda()       
    probs = bow(instance).cuda() # forward pass
    loss = loss_function(probs, label)
    loss.backward()
    opt.step()
  end = time.time()  
  print('epoch is: ' +str(epoch) + ' and loss is: ' +str(loss.data) + ' and time taken is: ' +str(end-start))